# ASR


Я скачала файлы через youtube-dl и перегнала аудио в wav через онлайн-конвертер

`youtube-dl --write-auto-sub -x --audio-format “wav” “https://www.youtube.com/watch?v=TzHTIVXdse4”`

In [1]:
!pip install -q SpeechRecognition

In [2]:
!pip install -q git+https://github.com/openai/whisper.git soundfile

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
!cd "drive/MyDrive/Colab Notebooks/data" && ls


 ground_truth.txt  'Two US Banks Just Failed - What Happened, and What Now-WowVQ4rhbt8.en.vtt'
 recognized_text   'Two US Banks Just Failed - What Happened, and What Now-WowVQ4rhbt8.wav'


In [5]:
import speech_recognition as sr

In [6]:
DATA_PATH = 'drive/MyDrive/Colab Notebooks/data/'
audio_fn = 'Two US Banks Just Failed - What Happened, and What Now-WowVQ4rhbt8.wav'
subtt_fn = 'Two US Banks Just Failed - What Happened, and What Now-WowVQ4rhbt8.en.vtt'

In [7]:
r = sr.Recognizer()
with sr.AudioFile(DATA_PATH + audio_fn) as source:
    audio = r.record(source)

In [23]:
results = dict()

In [24]:
# OpenAI Whisper library
results['whisper'] = r.recognize_whisper(audio, language='english')
results['whisper']

100%|███████████████████████████████████████| 139M/139M [01:19<00:00, 1.83MiB/s]


" Hey guys, it's Richard Washington, PlaneBaggle. It's been a crazy week in the world of finance. As you've probably heard by now, we've seen the collapse of not one, but two US banks over the span of just a couple of days this past week. It started on Wednesday with Silvergate Capital Corp, which is a crypto-friendly institution who stocked was already down 94% over the past six months. And now it's said it would begin its voluntary liquidation to try and meet its liabilities, which caused the stock to fall roughly another 50% since it's still a publicly traded position. And then Friday, after seeing its own stock dropped 60% on Wednesday alone, and another 66 plus percent in pre-market trading on Thursday before the stock trading was halted. Silicon Valley Bank, a commercial bank that served primarily startups and tech companies in California, was shut down by the FDIC, the Federal Deposit Insurance Corporation. And while the follow-up Silverbank is itself problematic, I can't emphas

In [25]:
with open(DATA_PATH + 'recognized_text/whisper.txt', 'w') as f:
    f.write(results['whisper'])

In [15]:
!pip install -q pocketsphinx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.2/29.2 MB 50.0 MB/s eta 0:00:00


In [ ]:
# Pocket Sphinx
results['pocketsphinx'] = r.recognize_sphinx(audio)
results['pocketsphinx']

In [8]:
with open(DATA_PATH + 'recognized_text/pocketsphinx.txt', 'w') as f:
    f.write(results['pocketsphinx'])

# HuggingFace transformers

In [9]:
!pip install -q transformers jiwer

Пример использования обученной модели для распознавания.

Где взять модели? https://huggingface.co/models?pipeline_tag=automatic-speech-recognition

In [4]:
from transformers import pipeline

In [5]:
transcriber = pipeline("automatic-speech-recognition", "facebook/wav2vec2-base-960h")
res = transcriber(DATA_PATH + audio_fn, chunk_length_s=10)

config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0']
You sho

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

In [14]:
with open(DATA_PATH + 'recognized_text/facebook.txt', 'w') as f:
    f.write(res['text'].lower())

## WER

Для эталона я скачала автоматические субтитры с ютуба и отредактировала (пришлось поменять совсем чуть-чуть)

In [10]:
from jiwer import wer

reference = "hello world"
hypothesis = "hello a duck"

error = wer(reference, hypothesis)

In [12]:
results = dict()

In [16]:
import os
import re

with open(DATA_PATH + 'ground_truth.txt', 'r') as f:
    ground_truth = f.read()

with open(DATA_PATH + subtt_fn, 'r') as f:
    yt = f.read()

results['youtube'] = yt.split('\n')[6]

for fn in os.listdir(DATA_PATH + 'recognized_text/'):
    with open(DATA_PATH + 'recognized_text/' + fn) as f:
        content = f.read().lower() # preprocess text
        content = re.sub('[^\w\s]', '', content)
        results[fn.split('.')[0]] = content

# compare to edited youtube subtitle, youtube wins
for asr in results:
    print(asr, wer(ground_truth, results[asr]), sep='\t')

youtube	0.00511770726714432
pocketsphinx	0.4278403275332651
facebook	0.24872057318321392
whisper	0.11941316956670078
